# IOT Detections

Lucho Oct| 2023 <br>
Tout Doucement by Feist

Objetivo: Clasificar registros de paquetes para predecir qué dispositivo los envió.

Número de muestras: 13,138

Número de clases (dispositivos):

    miscellaneous: 944

    camera: 8010

    assistant: 2178

    outlet: 1802

    mobile: 186

Número de características por registro: 18

Tipo de problema: Clasificación multiclase supervisada

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib_inline as plt

import tensorflow as tf
from numpy import genfromtxt
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import (accuracy_score, confusion_matrix,f1_score,precision_score, recall_score, classification_report, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Import Dataset

In [2]:
df = pd.read_csv("label_feature_IOT.csv")
df_raw = df.copy()
df.head()

,Label,IPLength,IPHeaderLength,TTL,Protocol,SourcePort,DestPort,SequenceNumber,AckNumber,WindowSize,TCPHeaderLength,TCPLength,TCPStream,TCPUrgentPointer,IPFlags,IPID,IPchecksum,TCPflags,TCPChecksum
0,TCP_Miscellaneous,60,20,64,6,51078,8883,0,0,29200,40,0,0,0,0x00004000,0x00001be9,0x00005b5e,0x00000002,0x0000ef50
1,TCP_Miscellaneous,60,20,64,6,51078,8883,0,0,29200,40,0,0,0,0x00004000,0x00001bea,0x00005b5d,0x00000002,0x0000e77c
2,TCP_Miscellaneous,60,20,64,6,51078,8883,0,0,29200,40,0,0,0,0x00004000,0x00001beb,0x00005b5c,0x00000002,0x0000d7d4
3,TCP_Miscellaneous,153,20,64,6,54031,443,1,1,7264,32,101,1,0,0x00004000,0x00007397,0x0000bf99,0x00000018,0x00005697
4,TCP_Miscellaneous,52,20,64,6,54031,443,102,86,7264,32,0,1,0,0x00004000,0x00007398,0x0000bffd,0x00000010,0x0000151c


In [3]:
df.shape

(13138, 19)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13138 entries, 0 to 13137
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Label                 13138 non-null  object
 1   IPLength              13138 non-null  int64 
 2   IPHeaderLength        13138 non-null  int64 
 3   TTL                   13138 non-null  int64 
 4              Protocol   13138 non-null  int64 
 5   SourcePort            13138 non-null  int64 
 6   DestPort              13138 non-null  int64 
 7   SequenceNumber        13138 non-null  int64 
 8   AckNumber             13138 non-null  int64 
 9   WindowSize            13138 non-null  int64 
 10  TCPHeaderLength       13138 non-null  int64 
 11  TCPLength             13138 non-null  int64 
 12  TCPStream             13138 non-null  int64 
 13  TCPUrgentPointer      13138 non-null  int64 
 14  IPFlags               13138 non-null  object
 15  IPID                  13138 non-null

In [5]:
df = df.drop_duplicates()

In [6]:
df.describe(include="all")

,Label,IPLength,IPHeaderLength,TTL,Protocol,SourcePort,DestPort,SequenceNumber,AckNumber,WindowSize,TCPHeaderLength,TCPLength,TCPStream,TCPUrgentPointer,IPFlags,IPID,IPchecksum,TCPflags,TCPChecksum
count,13127,13127.000000,13127.0,13127.000000,13127.0,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.000000,13127.0,13127,13127,13127,13127,13127
unique,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,11795,10867,7,11752
top,TCP_Camera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x00004000,0x00000000,0x0000def7,0x00000018,0x00002775
freq,8010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11666,344,33,8579,8
mean,NaN,120.504456,20.0,85.257789,6.0,43621.073436,1355.252152,93735.828826,3020.133846,4442.968233,31.396359,69.108098,48.926716,0.0,NaN,NaN,NaN,NaN,NaN
std,NaN,93.517570,0.0,60.071782,0.0,16338.927822,2557.625878,109572.836098,3068.655178,8512.927973,5.003737,94.379576,102.577287,0.0,NaN,NaN,NaN,NaN,NaN
min,NaN,40.000000,20.0,64.000000,6.0,3114.000000,80.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
25%,NaN,60.000000,20.0,64.000000,6.0,43930.500000,443.000000,147.000000,75.000000,1689.000000,32.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
50%,NaN,142.000000,20.0,64.000000,6.0,45739.000000,443.000000,33911.000000,2313.000000,2529.000000,32.000000,90.000000,2.000000,0.0,NaN,NaN,NaN,NaN,NaN
75%,NaN,142.000000,20.0,64.000000,6.0,55778.000000,443.000000,184307.000000,5403.000000,2529.000000,32.000000,90.000000,35.000000,0.0,NaN,NaN,NaN,NaN,NaN


### Parte 1

In [7]:
# Cargar características (features) 
feature = genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=(i for i in range(1,19)), dtype=int, skip_header=1)

# Cargar etiquetas (target) 
target = genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=(0), dtype=str, skip_header=1)

# Convertir las últimas 5 columnas (índices -5 a -1) de feature de hexadecimal a entero decimal
for c in range(-5, 0):
    for i in range(len(feature[:, c])):
        feature[:, c][i] = int(str(feature[:, c][i]), 16)

# Codificar las etiquetas de texto a números
labels = LabelEncoder().fit_transform(target)

# Escalar las características para que tengan media 0 y varianza 1
feature_std = StandardScaler().fit_transform(feature)

# Dividir datos en conjunto de entrenamiento (75%) y prueba (25%)
x_train, x_test, y_train, y_test = train_test_split(feature_std, labels, test_size=0.25, random_state=0)

print("Inicio:__________________________________")
###################################################
## Función para imprimir métricas estadísticas

def print_stats_metrics(y_test, y_pred):    
    print('Exactitud (Accuracy): %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Matriz de confusión")
    print(confmat)
    print(pd.crosstab(y_test, y_pred, rownames=['Verdadero'], colnames=['Predicho'], margins=True))
    print('Precisión (Precision): %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Exhaustividad (Recall): %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Medida F1 (F1-measure): %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

######################## Regresión Logística ##############################
print("######################## Regresión Logística ##############################")
clfLog = LogisticRegression()
clfLog.fit(x_train, y_train)
predictions = clfLog.predict(x_test)
print_stats_metrics(y_test, predictions)

######################## Bosque Aleatorio (Random Forest) ##############################
print("######################## Bosque Aleatorio ##############################")
clfRandForest = RandomForestClassifier()
clfRandForest.fit(x_train, y_train)
predictions = clfRandForest.predict(x_test)
print_stats_metrics(y_test, predictions)

####################### Árbol de Decisión #######################
print("####################### Árbol de Decisión #######################")
clfDT = DecisionTreeRegressor()
clfDT.fit(x_train, y_train)
predictions = clfDT.predict(x_test)
print_stats_metrics(y_test, predictions)

####################### Naive Bayes #######################
print("####################### Naive Bayes #######################")
clfNB = GaussianNB()
clfNB.fit(x_train, y_train)
predictions = clfNB.predict(x_test)
print_stats_metrics(y_test, predictions)

Inicio:__________________________________
######################## Regresión Logística ##############################
Exactitud (Accuracy): 0.97
Matriz de confusión
[[ 550    7    3    1    0]
 [  22 1978    0    0    0]
 [  41    0  170    0    0]
 [  29    4    4    3    0]
 [   0    0    0    0  473]]
Predicho     0     1    2  3    4   All
Verdadero                              
0          550     7    3  1    0   561
1           22  1978    0  0    0  2000
2           41     0  170  0    0   211
3           29     4    4  3    0    40
4            0     0    0  0  473   473
All        642  1989  177  4  473  3285
Precisión (Precision): 0.967
Exhaustividad (Recall): 0.966
Medida F1 (F1-measure): 0.962
######################## Bosque Aleatorio ##############################
Exactitud (Accuracy): 1.00
Matriz de confusión
[[ 561    0    0    0    0]
 [   0 2000    0    0    0]
 [   0    0  211    0    0]
 [   0    0    0   40    0]
 [   0    0    0    0  473]]
Predicho     0     1    

### Parte 2

In [8]:
tasa_aprendizaje = 0.01  # 0.1, 0.0001, etc
n_epocas = 10000


###############################################################
## tf.one_hot()

def convertirOneHot(datos):
    y_onehot = [0] * len(datos)
    for i, j in enumerate(datos):
        y_onehot[i] = [0] * (datos.max() + 1)
        y_onehot[i][j] = 1
    return y_onehot

###############################################################

feature = genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=(i for i in range(1,19)), dtype=int, skip_header=1)
for c in range(-5, 0):
    for i in range(len(feature[:, c])):
        feature[:, c][i] = int(str(feature[:, c][i]), 16)
target = genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=(0), dtype=str, skip_header=1)

###############################################################

sc = StandardScaler()
sc.fit(feature)
feature_normalizado = sc.transform(feature)

#############################################################

target_label = LabelEncoder().fit_transform(target)
target_onehot = convertirOneHot(target_label)

#############################################################

x_entrenamiento, x_prueba, y_entrenamiento_onehot, y_prueba_onehot = \
    train_test_split(feature_normalizado, target_onehot, \
                     test_size=0.25, random_state=0)

############################################################

A = x_entrenamiento.shape[1]
B = len(y_entrenamiento_onehot[0])
print(A)  # características
print(B)  # columnas
print("Inicio:__________________________________")

###############################################################
lista_precision = []
lista_exactitud = []

def imprimir_estadisticas_metricas(y_prueba, y_pred):
    print('Exactitud: %.2f' % accuracy_score(y_prueba, y_pred))
    #Exactitud: 0.84
    lista_exactitud.append(accuracy_score(y_prueba, y_pred))
    matriz_confusion = confusion_matrix(y_true=y_prueba, y_pred=y_pred)
    print("Matriz de confusión")
    print(matriz_confusion)
    print(pd.crosstab(y_prueba, y_pred, rownames=['Verdadero'], colnames=['Predicho'], margins=True))
    lista_precision.append(precision_score(y_true=y_prueba, y_pred=y_pred, average='weighted'))
    print('Precisión: %.3f' % precision_score(y_true=y_prueba, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_prueba, y_pred=y_pred, average='weighted'))
    print('Medida F1: %.3f' % f1_score(y_true=y_prueba, y_pred=y_pred, average='weighted'))

#####################################################################

def graficar_metrica_por_epoca():
    x_epocas = []
    y_epocas = []
    for i, val in enumerate(lista_exactitud):
        x_epocas.append(i)
        y_epocas.append(val)

    plt.scatter(x_epocas, y_epocas, s=50, c='lightgreen', marker='s', label='puntaje')
    plt.xlabel('época')
    plt.ylabel('puntaje')
    plt.title('Puntaje por época')
    plt.legend()
    plt.grid()
    plt.show()

18
5
Inicio:__________________________________


In [ ]:
# Parámetros
learning_rate = 0.01
n_epochs = 10000

# Función para convertir etiquetas a one-hot
def convertirOneHot(datos):
    y_onehot = np.zeros((len(datos), datos.max() + 1))
    for i, j in enumerate(datos):
        y_onehot[i, j] = 1
    return y_onehot

# Carga y preprocesamiento de datos (ajusta según tu CSV)
feature = np.genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=range(1,19), dtype=int, skip_header=1)
for c in range(-5, 0):
    for i in range(len(feature[:, c])):
        feature[i, c] = int(str(feature[i, c]), 16)

target = np.genfromtxt('label_feature_IOT.csv', delimiter=',', usecols=(0), dtype=str, skip_header=1)

sc = StandardScaler()
feature_normalizado = sc.fit_transform(feature)

target_label = LabelEncoder().fit_transform(target)
target_onehot = convertirOneHot(target_label)

# División train-test
x_train, x_test, y_train_onehot, y_test_onehot = train_test_split(
    feature_normalizado, target_onehot, test_size=0.25, random_state=0)

A = x_train.shape[1]
B = y_train_onehot.shape[1]
print(A)  # número de características
print(B)  # número de clases
print("Inicio:__________________________________")

# Definir modelo con tf.keras
modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(A,)),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(B, activation='softmax')
])

modelo.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Listas para almacenar métricas
accuracy_scores_list = []
precision_scores_list = []

def imprimir_estadisticas_metricas(y_true, y_pred):
    print('Exactitud: %.2f' % accuracy_score(y_true, y_pred))
    accuracy_scores_list.append(accuracy_score(y_true, y_pred))
    matriz_confusion = confusion_matrix(y_true, y_pred)
    print("Matriz de confusión")
    print(matriz_confusion)
    print(pd.crosstab(y_true, y_pred, rownames=['Verdadero'], colnames=['Predicho'], margins=True))
    precision = precision_score(y_true, y_pred, average='weighted')
    precision_scores_list.append(precision)
    print('Precisión: %.3f' % precision)
    print('Recall: %.3f' % recall_score(y_true, y_pred, average='weighted'))
    print('Medida F1: %.3f' % f1_score(y_true, y_pred, average='weighted'))

def graficar_metrica_por_epoca():
    x_epocas = list(range(len(accuracy_scores_list)))
    y_epocas = accuracy_scores_list

    plt.scatter(x_epocas, y_epocas, s=50, c='lightgreen', marker='s', label='puntaje')
    plt.xlabel('Época')
    plt.ylabel('Exactitud')
    plt.title('Exactitud por época')
    plt.legend()
    plt.grid()
    plt.show()

# Entrenamiento manual con evaluación por época
for i in range(n_epochs):
    print(f"Época {i+1} de {n_epochs}")
    modelo.fit(x_train, y_train_onehot, epochs=1, batch_size=32, verbose=0)

    # Evaluar
    y_pred_prob = modelo.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test_onehot, axis=1)

    imprimir_estadisticas_metricas(y_true, y_pred)

    if i == n_epochs - 1:
        graficar_metrica_por_epoca()

18
5
Inicio:__________________________________
Época 1 de 10000


C:\Users\Lupin III\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Exactitud: 0.61
Matriz de confusión
[[   0  561    0    0    0]
 [   0 2000    0    0    0]
 [   0  211    0    0    0]
 [   0   40    0    0    0]
 [   0  473    0    0    0]]
Predicho      1   All
Verdadero            
0           561   561
1          2000  2000
2           211   211
3            40    40
4           473   473
All        3285  3285
Precisión: 0.371
Recall: 0.609
Medida F1: 0.461
Época 2 de 10000


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Exactitud: 0.79
Matriz de confusión
[[   0  512   35    0   14]
 [   0 2000    0    0    0]
 [   0   53  134    0   24]
 [   0   32    5    0    3]
 [   0    0    0    0  473]]
Predicho      1    2    4   All
Verdadero                      
0           512   35   14   561
1          2000    0    0  2000
2            53  134   24   211
3            32    5    3    40
4             0    0  473   473
All        2597  174  514  3285
Precisión: 0.651
Recall: 0.794
Medida F1: 0.712
Época 3 de 10000


...

103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Exactitud: 0.97
Matriz de confusión
[[ 541    1   19    0    0]
 [   6 1994    0    0    0]
 [  17    0  194    0    0]
 [  32    2    6    0    0]
 [   0    0    0    0  473]]
Predicho     0     1    2    4   All
Verdadero                           
0          541     1   19    0   561
1            6  1994    0    0  2000
2           17     0  194    0   211
3           32     2    6    0    40
4            0     0    0  473   473
All        596  1997  219  473  3285
Precisión: 0.964
Recall: 0.975
Medida F1: 0.969
Época 19 de 10000


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Exactitud: 0.98
Matriz de confusión
[[ 539    1   20    0    1]
 [   6 1994    0    0    0]
 [  17    0  194    0    0]
 [  25    2   10    3    0]
 [   0    0    0    0  473]]
Predicho     0     1    2  3    4   All
Verdadero                              
0          539     1   20  0    1   561
1            6  1994    0  0    0  2000
2           17     0  194  0    0   211
3           25     2   10  3    0    40
4            0     0    0  0  473   473
All        587  1997  224  3  474  3285
Precisión: 0.976
Recall: 0.975
Medida F1: 0.971
Época 20 de 10000
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Exactitud: 0.97
Matriz de confusión
[[ 541    0   19    0    1]
 [   6 1994    0    0    0]
 [  17    0  194    0    0]
 [  32    1    7    0    0]
 [   0    0    0    0  473]]
Predicho     0     1    2    4   All
Verdadero                           
0          541     0   19    1   561
1            6  1994    0    0  2000
2           17     0  194    0

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Exactitud: 0.98
Matriz de confusión
[[ 540    2   19    0    0]
 [   6 1994    0    0    0]
 [  17    0  194    0    0]
 [  27    2    8    3    0]
 [   0    0    0    0  473]]
Predicho     0     1    2  3    4   All
Verdadero                              
0          540     2   19  0    0   561
1            6  1994    0  0    0  2000
2           17     0  194  0    0   211
3           27     2    8  3    0    40
4            0     0    0  0  473   473
All        590  1998  221  3  473  3285
Precisión: 0.976
Recall: 0.975
Medida F1: 0.971
Época 22 de 10000
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Exactitud: 0.98
Matriz de confusión
[[ 548    1   11    0    1]
 [   6 1994    0    0    0]
 [  17    0  194    0    0]
 [  26    2    9    3    0]
 [   0    0    0    0  473]]
Predicho     0     1    2  3    4   All
Verdadero                              
0          548     1   11  0    1   561
1            6  1994    0  0    0  2000
2           17    